<a href="https://colab.research.google.com/github/apoorvrocks/Machine-Learning-Assignment-1/blob/main/Assignment2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from PIL import Image
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import matplotlib.pyplot as plt

import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from pathlib import Path
import random
import os
import cv2

import torch.nn.functional as F

C:\Users\Asus\anaconda3\envs\venv_name\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Asus\anaconda3\envs\venv_name\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
class Layer(object):

    def forwardPass(self, x,train) :
        pass
    
    def backwardPass(self, grad_input) :
        pass

class ParamLayer(Layer):

    def update(self, learning_rate) :
        pass

In [ ]:
class LinearLayer(ParamLayer):
    
    def __init__(self,input_dim,output_dim,regularization,reg_rate=0.01):
        self.weights = np.random.normal(0.0, 0.1,size=input_dim * output_dim).reshape( input_dim,output_dim)
        self.weights = self.weights.astype('float64')
        self.biases = np.zeros((1, output_dim)).astype('float64')
        self.x = np.zeros(0)
        self.grad_biases = np.zeros(0)
        self.grad_weights = np.zeros(0)
        self.regularization = regularization
        self.reg_rate = reg_rate
        self.optimizer = Adam()
        
    def forwardPass(self,x,train = True):
        self.x = x 
        return np.dot(x,self.weights) + self.biases  
    
    def backwardPass(self,grad_input):
        self.grad_weights = np.dot(self.x.T,grad_input)
        self.grad_biases = np.sum(grad_input, axis=0, keepdims=True)
        return np.dot(grad_input,self.weights.T)
    
    def update(self, learning_rate):
        
        if self.regularization==None:
            grad_weights = self.grad_weights
        elif self.regularization == 'L1':
            grad_weights = self.grad_weights + self.reg_rate*np.sign(self.weights)
        elif self.regularization == 'L2':
            grad_weights = self.grad_weights + self.reg_rate*self.weights  
        self.weights,self.biases = self.optimizer.update(self.weights,grad_weights,self.biases,self.grad_biases)
        return self.weights,self.biases

In [ ]:
class SoftmaxLayer(Layer):

    def __init__(self):
        self.next_x = np.zeros(0)

    def forwardPass(self, x,train = True):
        e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return e_x / np.sum(e_x, axis=-1, keepdims=True)

    def backwardPass(self, grad_input):
        y = self.forwardPass(grad_input)
        return y*(1-y)

In [ ]:
class SigmoidLayer(Layer):

    def __init__(self):
        self.next_x = np.zeros(0)

    def forwardPass(self, x,train = True) :
        self.next_x = 1. / (1. + np.exp(-x))
        return self.next_x

    def backwardPass(self, grad_input):
        return (self.next_x * (1 - self.next_x)) * grad_input

In [ ]:
import math
class ReLU(Layer):
    def __init__(self):
        self.next_x = np.zeros(0)

    def forwardPass(self, x,train = True) :
        self.next_x = np.where(x >= 0, x, 0)
        return self.next_x

    def backwardPass(self, grad_input):
        return np.where(grad_input >= 0, 1, 0)

In [ ]:
class CrossEntropyLoss(Layer):

    def __init__(self, y):
        self.eps = 1e-3
        self.y = np.clip(y, self.eps, 1-self.eps)
        self.p = np.zeros(0)

    def forwardPass(self, x,train = True):
        self.p = np.clip(x, self.eps, 1.0 - self.eps)
        return -(self.y * np.log(self.p) + (1-self.y) * np.log(1 - self.p))

    def backwardPass(self, grad_input):
        error = (self.p - self.y) / (self.p - self.p ** 2)
        return error * grad_input

In [ ]:
class MSELayer(Layer):

    def __init__(self, y):
        self.y = y
        self.x = np.zeros(0)

    def forward(self, x, train = True) :
        self.x = x
        return np.square(x - self.y)

    def backward(self, grad_input):
        return -2 * (self.y - self.x) * grad_input

In [ ]:
class BatchNormLayer(ParamLayer):

    def __init__(self, dims) :
        self.gamma = np.ones((1, dims), dtype= 'float64')
        self.bias = np.zeros((1, dims), dtype= 'float64')
        self.eps = 1e-3

        # forward params
        self.var_x = np.zeros(0)
        self.stddev_x = np.zeros(0)
        self.x_minus_mean = np.zeros(0)
        self.standard_x = np.zeros(0)
        self.num_examples = 0
        self.mean_x = np.zeros(0)
        self.running_avg_gamma = 0.9
        self.running_mean = np.zeros(0)
        self.running_var = np.zeros(0)
        
        # backward params
        self.gamma_grad = np.zeros(0)
        self.beta_grad = np.zeros(0)
        self.optimizer = Adam()
        
    def forwardPass(self, x, train=True) :
        self.num_examples = x.shape[0]
        #print(f"in forward pass {self.num_examples}")
        if train:
            self.mean_x = np.mean(x, axis=0, keepdims=True)
            self.var_x = np.var(x , axis=0, keepdims=True)
            if np.array_equal(np.zeros(0),self.running_mean):
                self.running_mean = self.mean_x
                self.running_var = self.var_x
            else:
                self.running_mean = self.running_avg_gamma*self.running_mean + (1-self.running_avg_gamma)*self.mean_x
                self.running_var  = self.running_avg_gamma*self.running_var + (1-self.running_avg_gamma)*self.var_x
        else:
            self.mean_x = self.running_mean.copy()
            self.var_x = self.running_var.copy()

        self.var_x += self.eps
        self.stddev_x = np.sqrt(self.var_x)
        self.xmu = x - self.mean_x
        self.standard_x = self.xmu / self.stddev_x
        return self.gamma * self.standard_x + self.bias

    def backwardPass(self, grad_input) :
        
        standard_grad = grad_input * self.gamma

        var_grad = np.sum(standard_grad * self.xmu * -0.5 * self.var_x ** (-3/2),
                          axis=0, keepdims=True)
        stddev_inv = 1 / self.stddev_x
        aux_xmu = 2 * self.xmu / self.num_examples

        mean_grad = (np.sum(standard_grad * -stddev_inv, axis=0,
                            keepdims=True) +
                            var_grad * np.sum(-aux_xmu, axis=0,
                            keepdims=True))

        self.gamma_grad = np.sum(grad_input * self.standard_x, axis=0,
                                 keepdims=True)
        self.bias_grad = np.sum(grad_input, axis=0, keepdims=True)
        return standard_grad * stddev_inv + var_grad * aux_xmu + mean_grad / self.num_examples
               

    def update(self, learning_rate):
        
        self.gamma,self.bias = self.optimizer.update(self.gamma,self.gamma_grad,self.bias,self.bias_grad)

In [ ]:
class Dropout(Layer):
    
    def __init__(self,keep_prob):
        
        self.keep_prob = keep_prob
        
    def forwardPass(self,x,train):
        if train:
            self.mask = np.random.rand(x.shape[0],x.shape[1])
            self.mask = self.mask < self.keep_prob
            x = np.multiply(x,self.mask)
            x = x/self.keep_prob  
        
        return x
    
    def backwardPass(self,grad_input):

        return grad_input*self.mask

In [ ]:
class StochasticGradientDescent():
    def __init__(self, learning_rate=0.001, momentum=0):
        self.learning_rate = learning_rate 
        self.momentum = momentum
        self.w_updt = None
        self.b_updt = None
    def update(self, w, grad_wrt_w,b,grad_wrt_b):
        if self.w_updt is None:
            self.w_updt = np.zeros(np.shape(w))
        if self.b_updt is None:
            self.b_updt = np.zeros(np.shape(b))
        self.w_updt = self.momentum * self.w_updt + self.learning_rate * grad_wrt_w
        self.b_updt = self.momentum * self.b_updt + self.learning_rate * grad_wrt_b
        return w - self.w_updt, b - self.b_updt

In [ ]:
class Adam():
    
    def __init__(self, learning_rate= 3e-4, b1=0.9, b2=0.999):
        self.learning_rate = learning_rate
        self.eps = 1e-8
        self.m_w = None
        self.v_w = None
        self.m_b = None
        self.v_b = None
        # Decay rates
        self.b1 = b1
        self.b2 = b2

    def update(self, w, grad_wrt_w,b,grad_wrt_b):
        # If not initialized
        if self.m_w is None:
            self.m_w = np.zeros(np.shape(grad_wrt_w))
            self.v_w = np.zeros(np.shape(grad_wrt_w))
        if self.m_b is None:
            self.m_b = np.zeros(np.shape(grad_wrt_b))
            self.v_b = np.zeros(np.shape(grad_wrt_b))
        
        self.m_w = self.b1 * self.m_w + (1 - self.b1) * grad_wrt_w
        self.v_w = self.b2 * self.v_w + (1 - self.b2) * np.power(grad_wrt_w, 2)

        self.m_b = self.b1 * self.m_b + (1 - self.b1) * grad_wrt_b
        self.v_b = self.b2 * self.v_b + (1 - self.b2) * np.power(grad_wrt_b, 2)
        
        m_w_hat = self.m_w / (1 - self.b1)
        v_w_hat = self.v_w / (1 - self.b2)
        self.w_updt = self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.eps)
        
        m_b_hat = self.m_b / (1 - self.b1)
        v_b_hat = self.v_b / (1 - self.b2)

        self.b_updt = self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.eps)
        

        return w - self.w_updt,b-self.b_updt

In [ ]:
class CNN_Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        pass

    def backward(self, output_gradient, learning_rate):
        pass

In [ ]:
from scipy import signal

class Convolutional(CNN_Layer):
    def __init__(self, input_shape, kernel_size):
        input_depth, input_height, input_width = input_shape
        self.depth = input_depth
        self.input_shape = input_shape
        self.kernel_depth = input_depth
        self.output_shape = (input_depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (input_depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        # print('input', input.shape)
        # print('conv in', self.input_shape)
        # print('conv out', self.output_shape)
        for i in range(self.depth):
            for j in range(self.kernel_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)
        for i in range(self.depth):
            for j in range(self.kernel_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient

In [ ]:
class Dense(CNN_Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

In [ ]:
class Reshape(CNN_Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

In [ ]:
class Softmax(CNN_Layer):
    def forward(self, input):
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_gradient, learning_rate):
        n = np.size(self.output)
        return np.dot((np.identity(n) - self.output.T) * self.output, output_gradient)

In [ ]:
class Maxpool(CNN_Layer):
    def __init__(self, input_shape, kernel_size):
        input_depth, input_height, input_width = input_shape
        self.depth = input_depth
        self.input_shape = input_shape
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernel_size = kernel_size

    def forward(self, input):
        # print('max input', input.shape)
        # print('max in', self.input_shape)
        # print('max out', self.output_shape)
        output_conv = np.zeros(self.output_shape)
        self.maxargs = np.zeros((self.output_shape[0], self.output_shape[1], self.output_shape[2], 2))
        for d in range(self.depth):
            for i in range(self.input_shape[1] - self.kernel_size + 1):
                for j in range(self.input_shape[1] - self.kernel_size + 1):
                    mat = input[d, i:self.kernel_size + i, j:self.kernel_size+j]
                    indices = np.where(mat == mat.max())
                    try:
                        output_conv[d, i, j] = mat[indices[0][0], indices[1][0]]
                        self.maxargs[d, i, j] = [indices[0][0]+i, indices[1][0]+j]
                    except IndexError:
                        print(indices)
                        print(mat)
                        raise Exception("Nan in maxpool")
        return output_conv

    def backward(self, output_gradient, learning_rate):
        back_layer = np.zeros(self.input_shape)
        for d in range(self.depth):
            for i in range(self.input_shape[1] - self.kernel_size + 1):
                for j in range(self.input_shape[1] - self.kernel_size + 1):
                    ind = self.maxargs[d, i, j]
                    back_layer[d, int(ind[0]), int(ind[1])] = output_gradient[d, i, j]
        return back_layer

In [ ]:
class Avgpool(CNN_Layer):
    def __init__(self, input_shape, kernel_size):
        input_depth, input_height, input_width = input_shape
        self.depth = input_depth
        self.input_shape = input_shape
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernel_size = kernel_size

    def forward(self, input):
        output_conv = np.zeros(self.output_shape)
        for d in range(self.depth):
            for i in range(self.input_shape[1] - self.kernel_size + 1):
                for j in range(self.input_shape[1] - self.kernel_size + 1):
                    output_conv[d, i, j] = np.mean(input[d, i:self.kernel_size + i, j:self.kernel_size+j])
        return output_conv

    def backward(self, output_gradient, learning_rate):
        count_layer = np.zeros(self.input_shape)
        back_layer = np.zeros(self.input_shape)
        for d in range(self.depth):
            for i in range(self.input_shape[1] - self.kernel_size + 1):
                for j in range(self.input_shape[1] - self.kernel_size + 1):
                    count_layer[d, i:self.kernel_size + i, j:self.kernel_size+j] += 1
                    back_layer[d, i:self.kernel_size + i, j:self.kernel_size+j] += output_gradient[d, i, j]
        return back_layer / count_layer

In [ ]:
class CNN_Activation(CNN_Layer):
    def __init__(self, activation, activation_prime, *args):
        self.params = args
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input, self.params)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input, self.params))

In [ ]:
def sigmoid(x, args):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime( x, args):
    s = sigmoid(x, args)
    return s * (1 - s)

def tanh(x, args):
    return np.tanh(x)

def tanh_prime(x, args):
    return 1 - np.tanh(x) ** 2

def relu(x, args):
    return np.maximum(x, 0)

def relu_prime(x, args):
    return (x>0).astype(x.dtype)

def lkrelu(x, args):
    a = np.maximum(x, 0)
    b = np.minimum(x * args[0], 0)
    return a + b

def lkrelu_prime(x, args):
    a = (x>0).astype(x.dtype)
    b = (x<0).astype(x.dtype) * args[0]
    return a + b

In [ ]:
class Tanh(CNN_Activation):
    def __init__(self):
        super().__init__(tanh, tanh_prime)


class Sigmoid(CNN_Activation):
    def __init__(self):
        super().__init__(sigmoid, sigmoid_prime)


class Relu(CNN_Activation):
    def __init__(self):
        super().__init__(relu, relu_prime)


class LkRelu(CNN_Activation):
    def __init__(self, lk_const):
        super().__init__(lkrelu, lkrelu_prime, lk_const)

In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)


In [ ]:
class CNN:
    def __init__(self, layers):
        pass

    def predict(self, network, input):
        output = input
        for layer in network:
            output = layer.forward(output)
        return output

    def train(self, network, loss, loss_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, verbose = True):
        errors = []
        for e in range(epochs):
            error = 0
            for x, y in zip(x_train, y_train):
                # forward
                output = self.predict(network, x)
                
                # error
                error += loss(y, output)

                # backward
                grad = loss_prime(y, output)
                for layer in reversed(network):
                    grad = layer.backward(grad, learning_rate)

            error /= len(x_train)
            errors.append(error)
            if verbose:
                print(f"{e + 1}/{epochs}, error={error}")
        return errors

        

In [ ]:
class NeuralNetwork:

    def __init__(self, layers):
        self.layers = layers
        self.loss_layer = None
    
    def backward_propagation(self, output) :
        grad_input = self.loss_layer.backwardPass(np.ones_like(output)) 
        for layer_index in range(len(self.layers)-1, -1, -1):
            grad_input = self.layers[layer_index].backwardPass(grad_input)

    def update_params(self, learning_rate) :
        for layer_index in range(len(self.layers)):
            layer = self.layers[layer_index]
            if isinstance(layer, ParamLayer):
                layer.update(learning_rate)

    def evaluate_model(self, test_x, 
                       test_y):
        self.loss_layer = CrossEntropyLoss(test_y)
        #self.loss_layer = MSELayer(test_y)
        y_pred = self.predict(test_x)
        y_pred_labels = np.argmax(y_pred, axis=1)
        y_labels = np.argmax(test_y, axis=1)
        loss = self.loss_layer.forwardPass(y_pred)
        accuracy = float(np.sum(y_labels == y_pred_labels) / test_x.shape[0])
        return float(np.mean(loss)),y_pred_labels,accuracy

    def fit(self, x_train, y_train,
            learning_rate, steps,
            batch_size, x_test,
            y_test) :
        
        num_examples = x_train.shape[0]
        num_batches = math.ceil(num_examples/batch_size)
        n_epochs = steps//num_batches
        training_metrics = np.zeros((n_epochs,2))  # loss, accuracy
        validation_metrics = np.zeros((n_epochs,2))
        random_index = np.linspace(0, num_examples-1, num_examples).astype(int)
        for epoch in tqdm(range(n_epochs)):
            
            np.random.shuffle(random_index)
            x_train = x_train[random_index]
            y_train = y_train[random_index]
            for index_batch in range(0, num_examples, batch_size):
                mini_batch_x = x_train[index_batch: index_batch + batch_size]
                mini_batch_y = y_train[index_batch: index_batch + batch_size]
                self.loss_layer = CrossEntropyLoss(mini_batch_y)
                #self.loss_layer = MSELayer(mini_batch_y)
                y_pred = self.predict(mini_batch_x,train = True)
                self.loss_layer.forwardPass(y_pred)
                self.backward_propagation(y_pred)
                self.update_params(learning_rate)

            tr_loss,tr_pred,tr_accuracy = self.evaluate_model(x_train, y_train)
            val_loss,val_pred,val_accuracy = self.evaluate_model(x_test,y_test)
            if epoch%50==0 : 
                print(f"Epoch = {epoch} tr_loss = {tr_loss},tr_accuracy = {tr_accuracy} ,val_loss = {val_loss},val_accuracy={val_loss}")
            training_metrics[epoch,0] = tr_loss
            training_metrics[epoch,1] = tr_accuracy
            validation_metrics[epoch,0] = val_loss
            validation_metrics[epoch,1] = val_accuracy

        return training_metrics,validation_metrics

    def predict(self, x_test,train = False) :
        for layer in self.layers:
            x_test = layer.forwardPass(x_test,train)
        return x_test

In [ ]:
class Q1DataLoader():
    
    def __init__(self):
        self.data = np.load('../Assignment 1/data/pneumoniamnist.npz')
    
    def train_test_split(self):
        
        x_train = self.data['train_images'].reshape(self.data['train_images'].shape[0],-1).astype("float32")
        x_test = self.data['test_images'].reshape(self.data['test_images'].shape[0],-1).astype("float32")
        x_val = self.data['val_images'].reshape(self.data['val_images'].shape[0],-1).astype("float32")
        y_train = self.data['train_labels'].reshape((-1,))
        y_test = self.data['test_labels'].reshape((-1,))
        y_val = self.data['val_labels'].reshape((-1,))
        
        return x_train,y_train,x_test,y_test,x_val,y_val
    
    def one_hot(self,y):
        
        one_hot_y = np.zeros((y.size, y.max()+1)).astype("float32")
        one_hot_y[np.arange(y.size), y] = 1

        return one_hot_y 
    
    def get_metrics(self,pred,actual):
        
        n_correct_preds = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                if actual[i]==1:
                    tp += 1
                else:
                    tn += 1
            else:
                if actual[i]==1:
                    fn += 1
                else:
                    fp += 1

        accuracy = (tp+tn)/(tp+fp+tn+fn)
        if tp+fn==0:
            recall= 0 
        else:
            recall = tp/(tp+fn)
        if tp+fp==0:
            precision = 0
        else:   
            precision = tp/(tp+fp)
        if recall==0 and precision==0:
            F1 = 0 
        else:
            F1 = 2*recall*precision/(recall+precision)
        if tn+fp==0:
            specificity = 0
        else:
            specificity = tn/(tn+fp)
        AUC = (recall + specificity)/2

        return accuracy,F1,AUC

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

class Q2DataLoader():
    
    def __init__(self):
        self.data = np.load('../Assignment 1/data/bloodmnist.npz')
    
    def train_test_split(self):
        
        x_train = self.data['train_images'].reshape(self.data['train_images'].shape[0],-1).astype("float32")
        x_test = self.data['test_images'].reshape(self.data['test_images'].shape[0],-1).astype("float32")
        x_val = self.data['val_images'].reshape(self.data['val_images'].shape[0],-1).astype("float32")
        y_train = self.data['train_labels'].reshape((-1,))
        y_test = self.data['test_labels'].reshape((-1,))
        y_val = self.data['val_labels'].reshape((-1,))
    
        return x_train,y_train,x_test,y_test,x_val,y_val
    
    
    def confusion_mat(self,actual, pred):
        
        classes = 8
        mat = np.zeros((8, 8))
        for i, j in zip(actual, pred):
            mat[i][j] += 1
        return mat

    def get_metrics(self, y_prediction,y_true):
        
        y_true = y_true.reshape((-1,)).astype(np.int64)
        F1 = f1_score(y_true, y_prediction, average='macro')
        precision = precision_score(y_true, y_prediction, average='macro')
        recall = recall_score(y_true, y_prediction, average='macro')
        ACC = accuracy_score(y_true, y_prediction)
        AUC = (precision+recall)/2
        '''
        y_true = y_true.reshape((-1,)).astype(np.int64)
        y_prediction = y_prediction.astype(np.int64)
        cnf_matrix = confusion_mat(y_true, y_prediction)
        # print(cnf_matrix)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)

        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        div = TP+FN
        TPR = [0 if d==0 else 1/d for d in div]
        TPR = TPR * TP

        div = TN+FP
        TNR = [0 if d==0 else 1/d for d in div]
        TNR = TNR * TN

        div = TP+FP
        PPV = [0 if d==0 else 1/d for d in div]
        PPV = PPV * TP

        ACC = (TP+TN)/(TP+FP+FN+TN)
        AUC = (TNR + TPR)/2
        div = TPR + PPV
        F1 = [0 if d==0 else 1/d for d in div]
        F1 *= 2*TPR*PPV
        '''
        return ACC, F1, AUC


In [ ]:
class Q3Dataloader():
    
    def __init__(self,ann_path,im_path):
        
        self.ann_path = ann_path
        self.im_path = im_path
    
    def get_file_list(self,root, file_type):
        return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

    def get_train_df(self,ann_path, img_path):
    
        ann_path_list = self.get_file_list(self.ann_path, '.xml')
        ann = np.zeros((len(ann_path_list),4))
        for i in range(len(ann_path_list)):
            a_path = ann_path_list[i]
            root = ET.parse(a_path).getroot()
            ann[i][0] = int(root.find("./object/bndbox/xmin").text)
            ann[i][1] = int(root.find("./object/bndbox/ymin").text)
            ann[i][2] = int(root.find("./object/bndbox/xmax").text)
            ann[i][3] = int(root.find("./object/bndbox/ymax").text)
        return ann

    def get_image_data(self):
    
        image_list = get_file_list(self.im_path,'png')
        image_data = [ cv2.imread(image_path) for image_path in image_list]

        return image_data
    
    def resize_image_bounding_box(self):
        
        image_data = self.get_image_data()
        targetSize = (100,100)
        image_list = get_file_list(self.im_path,'png')
        resized_image_list = []
        for i in range(len(image_data)):
            x_scale = 100/image_data[i].shape[0]
            y_scale = 100/image_data[i].shape[1]
            train_box[i][0] = int(np.round(train_box[i][0]*x_scale))
            train_box[i][1] = int(np.round(train_box[i][1]*y_scale))
            train_box[i][2] = int(np.round(train_box[i][2]*x_scale))
            train_box[i][3] = int(np.round(train_box[i][3]*y_scale))
            image = cv2.imread(image_list[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) ## grayscale image
            norm_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) ## normalize
            image = cv2.resize(norm_image,targetSize)  ## resize to 100*100
            #print(image.shape)
            image = np.array(image)
            resized_image_list.append(image.ravel())

        return train_box,np.array(resized_image_list)  
    
    def train_test_split(self):
        
        train_box,resized_image_list = self.resize_image_bounding_box()
        return train_test_split(resized_image_list, train_box, test_size=0.3, random_state=34)
    
    def getMetrics(self,y_pred,actual):
        
        xA = max(y_pred[0], actual[0])
        yA = max(y_pred[1], actual[1])
        xB = min(y_pred[2], actual[2])
        yB = min(y_pred[3], actual[3])
        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (y_pred[2] - y_pred[0] + 1) * (y_pred[3] - y_pred[1] + 1)
        boxBArea = (actual[2] - actual[0] + 1) * (actual[3] - actual[1] + 1)
        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)
        # return the intersection over union value

        MSE = np.mean(0.5 * (actual - y_pred)**2)
        MAE = np.mean(np.abs(actual - y_pred))
        
        return MSE,MoU/y_pred.shape[0],MAE

In [ ]:
class Q4DataLoader():
    
    def __init__(self):
        
        self.phoneme = {}
        phoneme_list = ["bcl","dcl","gcl","pcl","tck","kcl","dcl","tcl","b","d","g","p","t","k","dx","q","jh","ch","s","sh","z","zh","f","th","v","dh","m","n","ng","em","en","eng","nx","l","r","w","y","hh","hv","el","iy","ih","eh","ey","ae","aa","aw","ay","ah","ao","oy","ow","uh","uw","ux","er","ax","ix","axr","ax-h","pau","epi","h#"]
        phonemlist_length = 63
        #create key value dictionary
        for ph in phoneme_list: 
            if('a' in ph or 'e' in ph or 'i' in ph or 'o' in ph or 'u' in ph):
                self.phoneme[ph] = 0                                                ##phoneme is vowel
            else:
                self.phoneme[ph] = 1
    
    def get_max_feature_len(self,x):
        
        max_len = 0
        n=x.__len__()
        for i in range(n):
            max_len = max(max_len,x[i].__len__())
        return max_len
    
    def add_padding(self,x,max_len):
    
        x_train = []
        n = x.__len__()
        for i in range(n):
            m=x[i].__len__()
            temp=np.zeros(max_len)
            if(m>max_len):
                temp=x[i][:max_len]
            else:
                temp[:m]=x[i]

            x_train.append(temp)
        return x_train

    def get_x_and_y(self,file_path):
    
        x = []
        y = []
        count = 0
        for folder in os.listdir(file_path):
            
            path = file_path + folder + "/"
            temp_name = ""
            for files in os.listdir(path):
                name = files.split(".")[0]
                if name != temp_name:
                    temp_name = name
                wav_file = path + name + ".WAV"
                phn_file = path + name + ".PHN"

                data, sampling_freq = librosa.load(wav_file,sr=None, mono=True,offset=0.0,duration=None)
                data=data.tolist()

                file_obj = open(phn_file, 'r')
                phonem_data = file_obj.readlines()
                n=np.shape(phonem_data)[0]

                for i in range(n):
                    
                    lower,upper,ph=phonem_data[i].split(" ")
                    lower = int(lower)
                    upper = int(upper)
                    ph = ph.replace("\n", "")
                    temp = data[lower:upper]
                    temp = np.array(temp)
                    count += 1
                    mfccs = librosa.feature.mfcc(temp, sr=sampling_freq)
                    mfccs = mfccs.flatten()
                    temp = mfccs.tolist()
                    x.append(temp)
                    y.append(self.phoneme[ph])
                    
        return x,y
    
    def preprocessing_audio(self,path):
  
        path_test  = path + "test/DR1/"
        path_train = path + "train/DR1/"
        x_train,y_train = self.get_x_and_y(path_train)         
        x_test,y_test = self.get_x_and_y(path_test)
        max_len = self.get_max_feature_len(x_train)
        max_len = max(max_len,self.get_max_feature_len(x_test))
        x_train = self.add_padding(x_train,max_len)
        x_test = self.add_padding(x_test,max_len)

        return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
    
    def get_metrics(self,pred,actual):
        n_correct_preds = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                if actual[i]==1:
                    tp += 1
                else:
                    tn += 1
            else:
                if actual[i]==1:
                    fn += 1
                else:
                    fp += 1
        
        return tp/actual.shape[0],fp/actual.shape[0],tn/actual.shape[0],fn/actual.shape[0]


In [ ]:
import os
import xml.etree.ElementTree as ET
import cv2

class TrafficDataLoader:
    
    def __init__(self,ann_path,im_path):
        
        self.ann_path = ann_path
        self.im_path = im_path
    
    def get_file_list(self,root, file_type):
        return [os.path.join(directory_path, f) for directory_path, directory_name, files in os.walk(root) for f in files if f.endswith(file_type)]

    def get_train_df(self,ann_path, img_path):
    
        ann_path_list = self.get_file_list(self.ann_path, '.xml')
        ann_path_list.sort()
        ann = np.zeros((len(ann_path_list),4))
        for i in range(len(ann_path_list)):
            a_path = ann_path_list[i]
            root = ET.parse(a_path).getroot()
            ann[i][0] = int(root.find("./object/bndbox/xmin").text)
            ann[i][1] = int(root.find("./object/bndbox/ymin").text)
            ann[i][2] = int(root.find("./object/bndbox/xmax").text)
            ann[i][3] = int(root.find("./object/bndbox/ymax").text)
        return ann

    def get_image_data(self):
    
        image_list = self.get_file_list(self.im_path,'png')
        image_list.sort()
        image_data = [ cv2.imread(image_path) for image_path in image_list]
        return image_data
    
    def resize_image_bounding_box(self):
        
        image_data = self.get_image_data()
        targetSize = (100,100)
        image_list = self.get_file_list(self.im_path,'png')
        resized_image_list = []
        train_box = self.get_train_df(self.ann_path, self.im_path)
        for i in range(len(image_data)):
            x_scale = 100/image_data[i].shape[1]
            y_scale = 100/image_data[i].shape[0]
            train_box[i][0] = int(np.round(train_box[i][0]*x_scale))
            train_box[i][1] = int(np.round(train_box[i][1]*y_scale))
            train_box[i][2] = int(np.round(train_box[i][2]*x_scale))
            train_box[i][3] = int(np.round(train_box[i][3]*y_scale))

            image = cv2.resize(image_data[i],targetSize)  ## resize to 100*100
            image = np.mean(image, axis=2)
            image = np.array(image)
            resized_image_list.append(image)

        return train_box,np.array(resized_image_list)  
    
    def split(self):
        
        train_box, resized_image_list = self.resize_image_bounding_box()
        print(train_box.shape, resized_image_list.shape)
        return train_test_split(resized_image_list, train_box, test_size=0.3, random_state=34)

In [ ]:
class MLP_Experiments():

    def __init__(self):

        pass

    def one_hot(self,y):
        
        one_hot_y = np.zeros((y.size, y.max()+1)).astype("float32")
        one_hot_y[np.arange(y.size), y] = 1

        return one_hot_y 

    def experiments(self,dataset):

        x_train,y_train,x_test,y_test,x_val,y_val = dataset.train_test_split()
        y_train_hot = self.one_hot(y_train)
        y_test_hot = self.one_hot(y_test)
        y_val_hot = self.one_hot(y_val)
    
        learning_rate = 1e-3
        batch_size = 100
        neurons = 100
        steps = 10000
        model_baseline = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                SigmoidLayer(),
                                LinearLayer(neurons, y_train_hot.shape[1],None),
                                SigmoidLayer()])

        baseline_tr_metrics,baseline_val_metrics = model_baseline.fit(x_train, y_train_hot, steps=steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               x_test = x_test,y_test = y_test_hot)
        baseline_tr_loss = baseline_tr_metrics[:,0]
        baseline_tr_accuracy = baseline_tr_metrics[:,1]
        baseline_val_loss = baseline_val_metrics[:,0]
        baseline_val_accuracy = baseline_val_metrics[:,1]
        baseline_pred = model_baseline.evaluate_model(x_test,y_test_hot)[1]

        print(f'baseline metrics on test data= {dataset.get_metrics(baseline_pred,y_test)}')

        model_batchNorm = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                    BatchNormLayer(neurons),
                                    SigmoidLayer(),
                                    LinearLayer(neurons, y_train_hot.shape[1],None),
                                    SigmoidLayer()])
        batchNorm_tr_metrics,batchNorm_val_metrics = model_batchNorm.fit(x_train, y_train_hot, steps=steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               x_test = x_test,y_test = y_test_hot)
        batchNorm_tr_loss = batchNorm_tr_metrics[:,0]
        batchNorm_tr_accuracy = batchNorm_tr_metrics[:,1]
        batchNorm_val_loss = batchNorm_val_metrics[:,0]
        batchNorm_val_accuracy = batchNorm_val_metrics[:,1]
        batchNorm_pred = model_batchNorm.evaluate_model(x_test,y_test_hot)[1]


        print(f'batchNorm metrics = {dataset.get_metrics(batchNorm_pred,y_test)}')

        model_dropout = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                    SigmoidLayer(),
                                    Dropout(0.8),
                                    LinearLayer(neurons, y_train_hot.shape[1],None),
                                    SigmoidLayer()])
        dropout_tr_metrics, dropout_val_metrics= model_dropout.fit(x_train, y_train_hot, steps=steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               x_test = x_test,y_test = y_test_hot)
        dropout_tr_loss = dropout_tr_metrics[:,0]
        dropout_tr_accuracy = dropout_tr_metrics[:,1]
        dropout_val_loss = dropout_val_metrics[:,0]
        dropout_val_accuracy = dropout_val_metrics[:,1]
        dropout_pred = model_dropout.evaluate_model(x_test,y_test_hot)[1]


        print(f'dropout metrics = {dataset.get_metrics(dropout_pred,y_test)}')

        model_L1 = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                    SigmoidLayer(),
                                    LinearLayer(neurons, y_train_hot.shape[1],None),
                                    SigmoidLayer()])
        L1_tr_metrics,L1_val_metrics = model_L1.fit(x_train, y_train_hot, steps=steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               x_test = x_test,y_test = y_test_hot)
        L1_tr_loss = L1_tr_metrics[:,0]
        L1_tr_accuracy = L1_tr_metrics[:,1]
        L1_val_loss = L1_val_metrics[:,0]
        L1_val_accuracy = L1_val_metrics[:,1]
        L1_pred = model_L1.evaluate_model(x_test,y_test_hot)[1]

        print(f'L1 metrics = {dataset.get_metrics(L1_pred,y_test)}')

        model_L2 = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                    SigmoidLayer(),
                                    LinearLayer(neurons, y_train_hot.shape[1],None),
                                    SigmoidLayer()])


        L2_tr_metrics,L2_val_metrics = model_L2.fit(x_train, y_train_hot, steps=steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               x_test = x_test,y_test = y_test_hot)
        L2_tr_loss = L2_tr_metrics[:,0]
        L2_tr_accuracy = L2_tr_metrics[:,1]
        L2_val_loss = L2_val_metrics[:,0]
        L2_val_accuracy = L2_val_metrics[:,1]
        L2_pred = model_L2.evaluate_model(x_test,y_test_hot)[1]

        print(f'L2 metrics = {dataset.get_metrics(L2_pred,y_test)}')

        num_batches = math.ceil(x_train.shape[0] / batch_size)
        steps_axis = np.linspace(1, steps/num_batches, len(baseline_val_loss)//10+1)
        steps_axis = steps_axis.astype("int")

        plt.plot(baseline_val_loss, label="Baseline")
        plt.plot(batchNorm_val_loss, label="BatchNorm")
        plt.plot(dropout_val_loss, label="Dropout")
        plt.plot(L1_val_loss,label = "L1")
        plt.plot(L2_val_loss,label = "L2")
        plt.ylabel("Validation Loss")
        plt.xlabel("Steps")
        plt.title("Loss versus Iterations")
        locs = np.linspace(0, len(baseline_val_loss)-1, len(baseline_val_loss)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid()
        plt.legend()
        plt.show()
        plt.close()

        plt.plot(baseline_tr_loss, label="Baseline")
        plt.plot(batchNorm_tr_loss, label="BatchNorm")
        plt.plot(dropout_tr_loss, label="Dropout")
        plt.plot(L1_tr_loss,label = "L1")
        plt.plot(L2_tr_loss,label = "L2")
        plt.ylabel("Training Loss")
        plt.xlabel("Steps")
        plt.title("Loss versus Iterations")
        locs = np.linspace(0, len(baseline_tr_loss)-1, len(baseline_tr_loss)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid()
        plt.legend()
        plt.show()
        plt.close()

        plt.plot(baseline_val_accuracy, label="Baseline")
        plt.plot(batchNorm_val_accuracy, label="BatchNorm")
        plt.plot(dropout_val_accuracy, label="Dropout")
        plt.plot(L1_val_accuracy,label = "L1")
        plt.plot(L2_val_accuracy,label = "L2")
        plt.ylabel("Validation Accuracy")
        plt.xlabel("Steps")
        plt.title("Accuracy versus iterations")
        locs = np.linspace(0, len(baseline_val_loss)-1, len(baseline_val_loss)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid() 
        plt.legend()
        plt.show()
        plt.close()

        plt.plot(baseline_tr_accuracy, label="Baseline")
        plt.plot(batchNorm_tr_accuracy, label="BatchNorm")
        plt.plot(dropout_tr_accuracy, label="Dropout")
        plt.plot(L1_tr_accuracy,label = "L1")
        plt.plot(L2_tr_accuracy,label = "L2")
        plt.ylabel("Training Accuracy")
        plt.xlabel("Steps")
        plt.title("Accuracy versus Iterations")
        locs = np.linspace(0, len(baseline_tr_accuracy)-1, len(baseline_tr_accuracy)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid()
        plt.legend()
        plt.show()
        plt.close()

        ## 2 layer neural Networks 


        model_baseline =NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                SigmoidLayer(),
                                LinearLayer(neurons, neurons,None),
                                SigmoidLayer(),
                                LinearLayer(neurons, y_train_hot.shape[1],None),
                                SigmoidLayer()])
        baseline_tr_metrics,baseline_val_metrics = model_baseline.fit(x_train, y_train_hot, steps=steps,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           x_test = x_test,y_test = y_test_hot)
        baseline_tr_loss = baseline_tr_metrics[:,0]
        baseline_tr_accuracy = baseline_tr_metrics[:,1]
        baseline_val_loss = baseline_val_metrics[:,0]
        baseline_val_accuracy = baseline_val_metrics[:,1]

        baseline_pred = model_baseline.evaluate_model(x_test,y_test_hot)[1]

        print(f'baseline metrics on test data = {dataset.get_metrics(baseline_pred,y_test)}')

        model_batchNorm = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                BatchNormLayer(neurons),
                                SigmoidLayer(),
                                LinearLayer(neurons, neurons,None),
                                BatchNormLayer(neurons),
                                SigmoidLayer(),
                                LinearLayer(neurons, y_train_hot.shape[1],None),
                                SigmoidLayer()])
        batchNorm_tr_metrics,batchNorm_val_metrics = model_batchNorm.fit(x_train, y_train_hot, steps=steps,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           x_test = x_test,y_test = y_test_hot)
        batchNorm_tr_loss = batchNorm_tr_metrics[:,0]
        batchNorm_tr_accuracy = batchNorm_tr_metrics[:,1]
        batchNorm_val_loss = batchNorm_val_metrics[:,0]
        batchNorm_val_accuracy = batchNorm_val_metrics[:,1]
        batchNorm_pred = model_batchNorm.evaluate_model(x_test,y_test_hot)[1]


        print(f'batchNorm metrics = {dataset.get_metrics(batchNorm_pred,y_test)}')

        model_dropout = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,None),
                                SigmoidLayer(),
                                Dropout(0.8),
                                LinearLayer(neurons, neurons,None),
                                SigmoidLayer(),
                                Dropout(0.6),
                                LinearLayer(neurons, y_train_hot.shape[1],None),
                                SigmoidLayer()])
        dropout_tr_metrics, dropout_val_metrics= model_dropout.fit(x_train, y_train_hot, steps=steps,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           x_test = x_test,y_test = y_test_hot)
        dropout_tr_loss = dropout_tr_metrics[:,0]
        dropout_tr_accuracy = dropout_tr_metrics[:,1]
        dropout_val_loss = dropout_val_metrics[:,0]
        dropout_val_accuracy = dropout_val_metrics[:,1]
        dropout_pred = model_dropout.evaluate_model(x_test,y_test_hot)[1]


        print(f'dropout metrics = {dataset.get_metrics(dropout_pred,y_test)}')

        model_L1 = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,'L1'),
                                SigmoidLayer(),
                                LinearLayer(neurons, neurons,'L1'),
                                SigmoidLayer(),
                                LinearLayer(neurons, y_train_hot.shape[1],'L1'),
                                SigmoidLayer()])
        L1_tr_metrics,L1_val_metrics = model_L1.fit(x_train, y_train_hot, steps=steps,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           x_test = x_test,y_test = y_test_hot)
        L1_tr_loss = L1_tr_metrics[:,0]
        L1_tr_accuracy = L1_tr_metrics[:,1]
        L1_val_loss = L1_val_metrics[:,0]
        L1_val_accuracy = L1_val_metrics[:,1]
        L1_pred = model_L1.evaluate_model(x_test,y_test_hot)[1]


        print(f'L1 metrics = {dataset.get_metrics(L1_pred,y_test)}')

        model_L2 = NeuralNetwork([LinearLayer(x_train.shape[1], neurons,'L2'),
                                SigmoidLayer(),
                                LinearLayer(neurons, neurons,'L2'),
                                SigmoidLayer(),
                                LinearLayer(neurons, y_train_hot.shape[1],'L2'),
                                SigmoidLayer()])


        L2_tr_metrics,L2_val_metrics = model_L2.fit(x_train, y_train_hot, steps=steps,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           x_test = x_test,y_test = y_test_hot)
        L2_tr_loss = L2_tr_metrics[:,0]
        L2_tr_accuracy = L2_tr_metrics[:,1]
        L2_val_loss = L2_val_metrics[:,0]
        L2_val_accuracy = L2_val_metrics[:,1]
        L2_pred = model_L2.evaluate_model(x_test,y_test_hot)[1]

        print(f'L2 metrics = {dataset.get_metrics(L2_pred,y_test)}')

        num_batches = math.ceil(x_train.shape[0] / batch_size)
        steps_axis = np.linspace(1, steps/num_batches, len(baseline_val_loss)//10+1)
        steps_axis = steps_axis.astype("int")

        plt.plot(baseline_val_loss, label="Baseline")
        plt.plot(batchNorm_val_loss, label="BatchNorm")
        plt.plot(dropout_val_loss, label="Dropout")
        plt.plot(L1_val_loss,label = "L1")
        plt.plot(L2_val_loss,label = "L2")
        plt.ylabel("Validation Loss")
        plt.xlabel("Steps")
        plt.title("Loss versus Iterations")
        locs = np.linspace(0, len(baseline_val_loss)-1, len(baseline_val_loss)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid()
        plt.legend()
        plt.show()
        plt.close()

        plt.plot(baseline_tr_loss, label="Baseline")
        plt.plot(batchNorm_tr_loss, label="BatchNorm")
        plt.plot(dropout_tr_loss, label="Dropout")
        plt.plot(L1_tr_loss,label = "L1")
        plt.plot(L2_tr_loss,label = "L2")
        plt.ylabel("Training Loss")
        plt.xlabel("Steps")
        plt.title("Loss versus Iterations")
        locs = np.linspace(0, len(baseline_tr_loss)-1, len(baseline_tr_loss)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid()
        plt.legend()
        plt.show()
        plt.close()

        plt.plot(baseline_val_accuracy, label="Baseline")
        plt.plot(batchNorm_val_accuracy, label="BatchNorm")
        plt.plot(dropout_val_accuracy, label="Dropout")
        plt.plot(L1_val_accuracy,label = "L1")
        plt.plot(L2_val_accuracy,label = "L2")
        plt.ylabel("Validation Accuracy")
        plt.xlabel("Steps")
        plt.title("Accuracy versus iterations")
        locs = np.linspace(0, len(baseline_val_loss)-1, len(baseline_val_loss)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid() 
        plt.legend()
        plt.show()
        plt.close()

        plt.plot(baseline_tr_accuracy, label="Baseline")
        plt.plot(batchNorm_tr_accuracy, label="BatchNorm")
        plt.plot(dropout_tr_accuracy, label="Dropout")
        plt.plot(L1_tr_accuracy,label = "L1")
        plt.plot(L2_tr_accuracy,label = "L2")
        plt.ylabel("Training Accuracy")
        plt.xlabel("Steps")
        plt.title("Accuracy versus Iterations")
        locs = np.linspace(0, len(baseline_tr_accuracy)-1, len(baseline_tr_accuracy)//10+1)
        #plt.xticks(locs, [str(x)+"k" for x in steps_axis])
        plt.grid()
        plt.legend()
        plt.show()
        plt.close()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, f1_score

class SVM_Experiments():
    
    def __init__(self):
        
        pass
    
    def run_SVM(self,X_train,x_test,y_train,y_test,C,kernel):
        classifier = SVC(kernel = kernel, C, random_state = 0)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        acc = accuracy_score(y_test, y_pred)
        
        return acc,f1,(precision+recall)/2
        
    def experiments(self,dataset):
        x_train,y_train,x_test,y_test,x_val,y_val = dataset.train_test_split()
        C = [0.01,1,5]
        kernels = ['linear','poly','rbf']
        for c in C:
            for kernel in kernels:
                acc,f1,auc = self.run_SVM(x_train,x_test,y_train,y_test,c,kernel)
                print(f"Accuracy ={acc} F1  = {f1} auc ={auc} for C= {c} kernel = {kernel}")
        
        
        

In [ ]:
class MnistResNet(nn.Module):
    def __init__(self,in_channels,n_classes):
        super(MnistResNet, self).__init__()

        self.model = models.resnet50(pretrained=True)

        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Change the output layer to output 10 classes instead of 1000 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, n_classes)

    def forward(self, x):
        return self.model(x)

class MnistVGG19(nn.Module):
    
    def __init__(self,in_channels,n_classes):
        super(MnistVGG19, self).__init__()
        self.model = models.vgg19(pretrained=True)
        in_features = self.model.classifier[-1].in_features
        self.model.classifier[-1] = nn.Linear(in_features, n_classes)

    def forward(self, x):
        return self.model(x)

    def forward(self, x):
        return self.model(x)


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8))
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)
    

In [ ]:
class Standard_CNN_Models():
    
    def __init__(self):
        pass
    
    def dataLoading(self,dataset):
        
        train_x,train_y,val_x,val_y,test_x,test_y = dataset.train_test_split()
        transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5 ), (0.5,0.5,0.5))])
        train_dataset = MyDataset(train_x, train_y, transform = transform)
        val_dataset = MyDataset(val_x, val_y, transform = transform) 
        test_dataset = MyDataset(test_x, test_y, transform = None) 
        self.train_dataloader = DataLoader(train_dataset, batch_size=64)
        self.test_dataloader = DataLoader(test_dataset, batch_size=64)
        self.val_dataloader = DataLoader(val_dataset, batch_size=64)
        
    def train_plots(self,model):
        
        loss_function = nn.CrossEntropyLoss()

        optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) 

        start_ts = time.time()

        losses = []
        batches = len(self.train_dataloader)
        val_batches = len(self.val_dataloader)
        AUC, F1, accuracy = [],[],[]

        for epoch in range(epochs):
            total_loss = 0
            progress = tqdm(enumerate(self.train_dataloader), desc="Loss: ", total=batches)
            model.train()

            for i, data in progress:
                X, y = data[0].to(device), data[1].to(device)
                model.zero_grad()
                outputs = model(X)
                loss = loss_function(outputs, y)
                loss.backward()
                optimizer.step()

                current_loss = loss.item()
                total_loss += current_loss
                progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            val_losses = 0
            AUC_l, f1_l, accuracy_l = [], [], []
            model.eval()
            with torch.no_grad():
                for i, data in enumerate(self.val_dataloader):
                    X, y = data[0].to(device), data[1].to(device)

                    outputs = model(X)

                    val_losses += loss_function(outputs, y)

                    predicted_classes = torch.max(outputs, 1)[1] 

                    a,f1,auc = Q1DataLoader().get_metrics(y.cpu(),predicted_classes.cpu())
                    AUC_l.append(auc)
                    f1_l.append(f1)
                    accuracy_l.append(a)

            print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
            print(f"AUC = {sum(AUC_l)/len(AUC_l)},f1 = {sum(f1_l)/len(f1_l)},accuracy = {sum(accuracy_l)/len(accuracy_l)}")
            AUC.append(sum(AUC_l)/len(AUC_l))
            accuracy.append(sum(accuracy_l)/len(accuracy_l))
            F1.append(sum(f1_l)/len(f1_l))
            losses.append(total_loss/batches) # for plotting learning curve

            print(f"Training time: {time.time()-start_ts}s")
            plt.plot(np.arange(0, len(losses)),losses)
            plt.ylabel("Training Loss")
            plt.xlabel("Steps")
            plt.title("Loss versus Iterations")
            plt.show()
            plt.close()

            plt.plot(np.arange(0, len(losses)),accuracy,label = "accuracy")
            plt.plot(np.arange(0, len(losses)),F1,label = "F1")
            plt.plot(np.arange(0, len(losses)),AUC,label = "AUC")
            plt.ylabel("Metrics")
            plt.xlabel("Steps")
            plt.title("Metrics versus Iterations")
            plt.legend()
            plt.show()
            plt.close()
        
    def experiments(self,dataset,in_channels,n_classes):
        
        if torch.cuda.is_available():
            device = torch.device("cuda") 
        else:
            device = torch.device("cpu")
        self.dataLoading(dataset)
        
        model = MnistResNet(in_channels,n_classes).to(device)
        self.train_plots(model)
        model = MnistVGG19(in_channels,n_classes).to(device)
        self.train_plots(model)


In [ ]:
class CNN_Experiments():

    def __init__(self):
        pass

    def plot_error(self, lkrelu, tan, sig, title_name):
        x = range(1, 21)
        
        plt.plot(x, lkrelu, label = "lkrelu")
        plt.plot(x, tan, label = "tan")
        plt.plot(x, sig, label = "sig")

        plt.xlabel('iteration')

        plt.ylabel('training error')
        plt.xticks(np.arange(0, 21, 5.0))
        plt.legend()

        plt.title(title_name)

        plt.show()


    def get_metrics(self, y_true, y_prediction):
        cnf_matrix = confusion_mat(y_true, y_prediction)
        # print(cnf_matrix)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)

        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        div = TP+FN
        TPR = [0 if d==0 else 1/d for d in div]
        TPR = TPR * TP

        div = TN+FP
        TNR = [0 if d==0 else 1/d for d in div]
        TNR = TNR * TN

        div = TP+FP
        PPV = [0 if d==0 else 1/d for d in div]
        PPV = PPV * TP

        ACC = (TP+TN)/(TP+FP+FN+TN)
        AUC = (TNR + TPR)/2
        div = TPR + PPV
        F1 = [0 if d==0 else 1/d for d in div]
        F1 *= 2*TPR*PPV

        return ACC, F1, AUC

    def confusion_mat(self, actual, pred):
        classes = 8
        mat = np.zeros((8, 8))
        
        for i, j in zip(actual, pred):
            mat[i][j] += 1
        return mat

    def preprocess_data(self, x, y, no_classes, limit=None):
        x = x[:limit]
        y = y[:limit]
        x = x.reshape(len(x), 3, 28, 28)
        x = x.astype("float32") / 255
        y = np_utils.to_categorical(y)
        y = y.reshape(len(y), no_classes, 1)
        return x, y

    def bb_intersection_over_union(self, boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

        iou = interArea / float(boxAArea + boxBArea - interArea)
        return np.abs(iou)
    def getMetrics(self, y_pred, actual):
            
        MoU = 0
        for i in range(y_pred.shape[0]):
            MoU += self.bb_intersection_over_union(y_pred[i], actual[i])
            
        MSE = np.mean(0.5 * (actual - y_pred)**2)
        MAE = np.mean(np.abs(actual - y_pred))

        return MSE,MoU/y_pred.shape[0],MAE


    def experiments(self, dataset):
        dataset = Q2DataLoader()
        X_train, Y_train, X_test, Y_test, X_val, Y_val = dataset.train_test_split()

        no_classes = 8
        x_train, y_train = self.preprocess_data(X_train, Y_train, no_classes)
        x_test, y_test = self.preprocess_data(X_test, Y_test, no_classes)
        x_val, y_val = self.preprocess_data(X_val, Y_val, no_classes)
        
        depth = 3
        img_size = 28
        kernel_size = 3
        l1_size = img_size - kernel_size + 1
        l2_size = l1_size - kernel_size + 1
        l3_size = l2_size - kernel_size + 1
        l4_size = l3_size - kernel_size + 1
        no_class = 8

        lr = 0.0001

        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            Softmax()
        ]
        cnn = CNN()
        th = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            Softmax()
        ]
        sig = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            Softmax()
        ]
        lrelu = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)

        self.plot_error(lrelu, th, sig, 'MAX POOLING')

        size = 3000

        op = []
        for x in x_test[:size]:
            op.append(cnn.predict( network, x))

        true_1hot = np.argmax(y_test[:size], axis = 1).reshape((size,))
        pred_1hot = np.argmax(op, axis = 1).reshape((size,))
        metr = self.get_metrics(pred_1hot, true_1hot)

        print('acc:', list(metr[0]))
        print('F1:', list(metr[1]))
        print('AUC:', list(metr[2]),'\n')


        
        
        
       
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            Softmax()
        ]
        th = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            Softmax()
        ]
        sig = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            Softmax()
        ]
        lrelu = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)

        self.plot_error(lrelu, th, sig, 'AVG POOLING')

        size = 3000

        op = []
        for x in x_test[:size]:
            op.append(cnn.predict( network, x))

        true_1hot = np.argmax(y_test[:size], axis = 1).reshape((size,))
        pred_1hot = np.argmax(op, axis = 1).reshape((size,))
        metr = self.get_metrics(pred_1hot, true_1hot)

        print('acc:', list(metr[0]))
        print('F1:', list(metr[1]))
        print('AUC:', list(metr[2]),'\n')


        










    
        
        dataset = Q1DataLoader()
        X_train, Y_train, X_test, Y_test, X_val, Y_val = dataset.train_test_split()

        no_classes = 2
        x_train, y_train = self.preprocess_data(X_train, Y_train, no_classes)
        x_test, y_test = self.preprocess_data(X_test, Y_test, no_classes)
        x_val, y_val = self.preprocess_data(X_val, Y_val, no_classes)
        
        depth = 1
        img_size = 28
        kernel_size = 3
        l1_size = img_size - kernel_size + 1
        l2_size = l1_size - kernel_size + 1
        l3_size = l2_size - kernel_size + 1
        l4_size = l3_size - kernel_size + 1
        no_class = 2

        lr = 0.0001

        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
            
        ]
        cnn = CNN()
        thm = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        thb = cnn.train(network, binary_cross_entropy, binary_cross_entropy_prime, x_train, y_train, epochs=20, learning_rate=lr)

        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        sigm = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        sigb = cnn.train(network, binary_cross_entropy, binary_cross_entropy_prime, x_train, y_train, epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        lrelum = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        lrelub = cnn.train(network, binary_cross_entropy, binary_cross_entropy_prime, x_train, y_train, epochs=20, learning_rate=lr)

        # self.plot_error(lrelu, th, sig, 'MAX POOLING')
        # lrelum = cnn.train(network, mse, mse_prime, x_train, y_train , epochs=20, learning_rate=lr)
        self.plot_error(lrelum, thm, sigm, 'MSE MAX POOLING')
        self.plot_error(lrelub, thb, sigb, 'BCE MAX POOLING')

        size = 3000

        op = []
        for x in x_test[:size]:
            op.append(cnn.predict( network, x))

        true_1hot = np.argmax(y_test[:size], axis = 1).reshape((size,))
        pred_1hot = np.argmax(op, axis = 1).reshape((size,))
        metr = self.get_metrics(pred_1hot, true_1hot)

        print('acc:', list(metr[0]))
        print('F1:', list(metr[1]))
        print('AUC:', list(metr[2]),'\n')


        
        
        
       
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        thm = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        thb = cnn.train(network, binary_cross_entropy, binary_cross_entropy_prime, x_train, y_train, epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        sigm = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        sigb = cnn.train(network, binary_cross_entropy, binary_cross_entropy_prime, x_train, y_train, epochs=20, learning_rate=lr)

        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        lrelum = cnn.train(network, mse, mse_prime, x_train, y_train , epochs=20, learning_rate=lr)
        lrelub = cnn.train(network, binary_cross_entropy, binary_cross_entropy_prime, x_train, y_train, epochs=20, learning_rate=lr)
        self.plot_error(lrelum, thm, sigm, 'MSE AVG POOLING')
        self.plot_error(lrelub, thb, sigb, 'BCE AVG POOLING')

        size = 3000

        op = []
        for x in x_test[:size]:
            op.append(cnn.predict( network, x))

        true_1hot = np.argmax(y_test[:size], axis = 1).reshape((size,))
        pred_1hot = np.argmax(op, axis = 1).reshape((size,))
        metr = self.get_metrics(pred_1hot, true_1hot)

        print('acc:', list(metr[0]))
        print('F1:', list(metr[1]))
        print('AUC:', list(metr[2]),'\n')




        '''TRAFFIC DATASET'''
        ann_path = '/content/drive/MyDrive/IISc/PRNN/traffic_data/annotations'
        image_path = '/content/drive/MyDrive/IISc/PRNN/traffic_data/images'
        dataset = TrafficDataLoader(ann_path, image_path)
        X_train, X_test, Y_train, Y_test = dataset.split()

        no_classes = 4
        x_train, y_train = self.preprocess_data(X_train, Y_train, no_classes)
        x_test, y_test = self.preprocess_data(X_test, Y_test, no_classes)
        
        depth = 1
        img_size = 100
        kernel_size = 3
        l1_size = img_size - kernel_size + 1
        l2_size = l1_size - kernel_size + 1
        l3_size = l2_size - kernel_size + 1
        l4_size = l3_size - kernel_size + 1
        no_class = 4

        lr = 0.0001

        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        cnn = CNN()
        th = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        sig = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            Maxpool((depth, l1_size, l1_size), kernel_size),
            # Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            Maxpool((depth, l3_size, l3_size), kernel_size),
            # Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        lrelu = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)

        self.plot_error(lrelu, th, sig, 'MAX POOLING')


        op = []
        for x in x_test:
            op.append(predict( network, x))
        metr = self.getMetrics(np.array(op).reshape(264, 4)*100, y_test.reshape(264, 4)*100)
        print(metr)


        
        
        
       
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            # Sigmoid(),
            Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        th = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            # LkRelu(0.1),
            Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        sig = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)
        
        network = [
            Convolutional((3, 28, 28), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l1_size, l1_size), kernel_size),
            Avgpool((depth, l1_size, l1_size), kernel_size),

            Convolutional((3, 24, 24), kernel_size),
            LkRelu(0.1),
            # Sigmoid(),
            # Tanh(),
            # Maxpool((depth, l3_size, l3_size), kernel_size),
            Avgpool((depth, l3_size, l3_size), kernel_size),
            
            Reshape((depth, l4_size, l4_size), (depth * l4_size * l4_size, 1)),
            Dense(depth * l4_size * l4_size, no_class),
            Sigmoid(),
        ]
        lrelu = cnn.train(network, mse, mse_prime, x_train , y_train , epochs=20, learning_rate=lr)

        self.plot_error(lrelu, th, sig, 'AVG POOLING')

        size = 3000


        op = []
        for x in x_test:
            op.append(predict( network, x))
        metr = self.getMetrics(np.array(op).reshape(264, 4)*100, y_test.reshape(264, 4)*100)
        print(metr)


In [ ]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 

class Predictor(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.2):
        super(Predictor, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.input_size = input_size
        
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.linear1 = nn.Linear(hidden_dim, output_size)
        # self.linear2 = nn.Linear(64, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        #x = x.long()
        #embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.linear1(out)
        # out = self.linear2(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        #print(weight)
        hidden = ((weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()).to(torch.float32),
                      (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()).to(torch.float32))
        #print(hidden)
        return hidden

class LSTM_experiment():
    def __init__(self):
        pass
    
    def lstm_pytorch(self,dataset):
      Xtrain, Ytrain, Xtest, Ytest, Xval, yval = dataset.train_test_split()

      padding_size=np.shspe(Xtrain)[2]  
      X_train = torch.from_numpy(Xtrain).float()
      Y_train = torch.from_numpy(Ytrain).float()

      X_test = torch.from_numpy(Xtest).float()
      Y_test = torch.from_numpy(Ytest).float()

      train_data = TensorDataset(X_train, Y_train)
      test_data = TensorDataset(X_test, Y_test)

      batch_size = 20

      train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
      test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

      output_size = 1

      #embedding_dim = 400
      hidden_dim = 64
      n_layers = 4
      input_size = 15

      model = Predictor(input_size, output_size, hidden_dim, n_layers)
      #model.to(device)

      lr=0.002
      #criterion = nn.BCELoss()
      criterion = nn.MSELoss()
      # criterion=nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=3e-2)

      epochs = 20
      counter = 0
      print_every = 1000
      clip = 5
      valid_loss_min = np.Inf
      losses = []
      model.train()
      accuracy_list=[]
      for i in range(epochs):
          h = model.init_hidden(batch_size)
          # h[0] = h[0].to(torch.float32)
          for inputs, labels in train_loader:
              counter += 1
              h = tuple([e.data for e in h])
              #inputs = inputs.to(torch.float64)
              # print(h)
              # print(len(h))
              # inputs, labels = inputs, labels.to(device)

              model.zero_grad()
              output, h = model(inputs, h)
              labels = labels.squeeze(-1)

              # print(output)
              loss = criterion(output, labels)
              loss.backward()
              nn.utils.clip_grad_norm_(model.parameters(), clip)
              optimizer.step()
              
              if counter%print_every == 0:
                  val_h = model.init_hidden(batch_size)
                  val_losses = []
                  num_correct = 0
                  num_yess = 0 
                  model.eval()
                  for inp, lab in test_loader:
                      val_h = tuple([each.data for each in val_h])
                      # inp, lab = inp.to(device), lab.to(device)
                      #print(val_h[0].size)
                      out, val_h = model(inp, val_h)
                      # lab = lab.squeeze(-1)
                      val_loss = criterion(out, lab.float())
                      #val_loss = criterion(out.squeeze(), lab.float())
                      val_losses.append(val_loss.item())
                      pred = torch.round(output.squeeze())  # Rounds the output to 0/1
                      
                      correct_tensor = pred.eq(labels.float().view_as(pred))
                      correct = np.squeeze(correct_tensor.cpu().numpy())
                      # pred = pred.squeeze(-1)
                      temp=pred.detach()
                      # print(lab,temp)
                      if ((lab.numpy()==temp.numpy()).all()):
                        yess = 1
                        num_yess += 1
                      else:
                        yess = 0
                      num_correct += np.sum(correct)
                      
                  model.train()
                  print("Epoch: {}/{}...".format(i+1, epochs),
                        "Step: {}...".format(counter),
                        "Loss: {:.6f}...".format(loss.item()),
                        "Val Loss: {:.6f}".format(np.mean(val_losses)))
                  test_acc = num_correct/len(test_loader.dataset)
                  print("Test accuracy: {:.3f}%".format(test_acc*100))
                  accuracy_list.append(test_acc*100)
                  if np.mean(val_losses) <= valid_loss_min:
                      torch.save(model.state_dict(), './state_dict.pt')
                      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                      valid_loss_min = np.mean(val_losses)
        
      f = plt.figure()
      plt.plot(accuracy_list)
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.legend()
      plt.show()


    
    def experiments(self,dataset):
      X_train, Y_train, Xtest, Ytest, Xval, yval = dataset.train_test_split()

      padding_size=np.shspe(X_train)[2]    

      # X_train, Xval, Y_train, yval = train_test_split(Xtrain, Ytrain, test_size=0.1, random_state=123)
      print(type(X_train))

      X_train=tf.convert_to_tensor(X_train)
      Y_train=tf.convert_to_tensor(Y_train)

      X_val=tf.convert_to_tensor(Xval)
      Y_val=tf.convert_to_tensor(yval)

      X_test=tf.convert_to_tensor(Xtest)
      Y_test=tf.convert_to_tensor(Ytest)

      
      input_shape=(20,padding_size)
      model = keras.Sequential()
      model.add(LSTM(128,input_shape=input_shape))
      model.add(Dropout(0.2))
      model.add(Dense(128, activation='relu'))
      model.add(Dense(64, activation='relu'))
      model.add(Dropout(0.4))
      model.add(Dense(48, activation='relu'))
      model.add(Dropout(0.4))
      model.add(Dense(24, activation='softmax'))
      model.summary()
      model.compile(optimizer='adam',loss='SparseCategoricalCrossentropy',metrics=['acc'])

      history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(Xval, Y_val), shuffle=False)

      history_dict=history.history
      loss_values=history_dict['loss']
      acc_values=history_dict['acc']
      val_loss_values = history_dict['val_loss']
      val_acc_values=history_dict['val_acc']
      epochs=range(1,51)
      fig,(ax1,ax2)=plt.subplots(1,2,figsize=(30,10))
      ax1.plot(epochs,loss_values,label='Training Loss')
      ax1.plot(epochs,val_loss_values, label='Validation Loss')
      ax1.set_title('Training and validation loss')
      ax1.set_xlabel('Epochs')
      ax1.set_ylabel('Loss')
      ax1.legend()
      ax2.plot(epochs,acc_values, label='Training accuracy')
      ax2.plot(epochs,val_acc_values,label='Validation accuracy')
      ax2.set_title('Training and validation accuracy')
      ax2.set_xlabel('Epochs')
      ax2.set_ylabel('Accuracy')
      ax2.legend()
      plt.show()

      TrainLoss, Trainacc = model.evaluate(X_train,Y_train)
      TestLoss, Testacc = model.evaluate(X_test, Y_test)
      y_pred=model.predict(X_test)

      print('Confusion_matrix: ',tf.math.confusion_matrix(Y_test, np.argmax(y_pred,axis=1)))
      

In [ ]:

if __name__=='__main__':
    
    dataset = Q1DataLoader()
    mlp_experiments = MLP_Experiments()
    mlp_experiments.experiments(dataset)
    Standard_CNN_Models().experiments(dataset,1,2)
    svm_experiments = SVM_Experiments()
    svm_experiments.experiments(dataset)
    
    dataset = Q2DataLoader()
    mlp_experiments = MLP_Experiments()
    mlp_experiments.experiments(dataset)
    Standard_CNN_Models().experiments(dataset,3,8)
    
    dataset = Q3DataLoader()
    Standard_CNN_Models().experiments(dataset,3,4)
    
    dataset = Q4DataLoader()
    mlp_experiments = MLP_Experiments()
    mlp_experiments.experiments(dataset)

    cnn_exp = CNN_Experiments()
    cnn_exp.experiments()

    dataset = Q4DataLoader()
    lstm_experiments = LSTM_experiment()
    lstm_experiments.experiments(dataset)

  0%|                                                                                          | 0/208 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_9376\2614723586.py:7: RuntimeWarning: overflow encountered in exp
  self.next_x = 1. / (1. + np.exp(-x))
  0%|▍                                                                                 | 1/208 [00:01<03:52,  1.12s/it]

[1 1 1 ... 1 1 1]
0.5489054209279931 0.7421410365335599 0.6493750472315728 0.625


 25%|███████████████████▊                                                             | 51/208 [00:54<02:43,  1.04s/it]

[1 1 1 ... 1 1 0]
0.27969717930593463 0.9207731520815633 0.44525289934853346 0.8221153846153846


 49%|██████████████████████████████████████▊                                         | 101/208 [01:46<02:00,  1.12s/it]

...

  0%|▍                                                                                 | 1/208 [00:00<01:56,  1.78it/s]

[1 1 0 ... 1 1 1]
0.3671343718773593 0.9182242990654206 0.4612694318644604 0.8301282051282052


 25%|███████████████████▊                                                             | 51/208 [00:28<01:32,  1.70it/s]

[1 0 1 ... 1 1 1]
0.10112044258356785 0.9683517417162277 0.4387186270579116 0.8557692307692307


 30%|████████████████████████▏                                                        | 62/208 [00:34<01:17,  1.88it/s]

In [ ]:
print(np.zeros(0))